In [ ]:
import cv2
import os
import csv
import numpy as np
import mediapipe as mp
from scipy.spatial import distance as dist

In [ ]:
# EAR Calculation


def eye_aspect_ratio(eye_points):
    A = dist.euclidean(eye_points[1], eye_points[5])
    B = dist.euclidean(eye_points[2], eye_points[4])
    C = dist.euclidean(eye_points[0], eye_points[3])
    return (A + B) / (2.0 * C)

# Indices for left and right eyes in MediaPipe FaceMesh
LEFT_EYE_IDX = [33, 160, 158, 133, 153, 144]
RIGHT_EYE_IDX = [362, 385, 387, 263, 373, 380]

# Paths to folders containing full-face images for drowsy and not drowsy classes
dataset_folders = {
    1: "", # Path to drowsy
    0: "" #Path to non drowsy
}

output_csv = "ear_features_fullface.csv"

mp_face_mesh = mp.solutions.face_mesh

with open(output_csv, mode='w', newline='') as file:
    writer = csv.writer(file)
    # CSV header (filename, label, avg_EAR)
    writer.writerow(["filename", "label", "avg_EAR"])

    with mp_face_mesh.FaceMesh(static_image_mode=True, max_num_faces=1, refine_landmarks=True) as face_mesh:
        for label, folder in dataset_folders.items():
            print(f"Processing label {label}, folder: {folder}")
            for filename in os.listdir(folder):
                if not filename.lower().endswith((".jpg", ".jpeg", ".png")):
                    continue

                img_path = os.path.join(folder, filename)
                image = cv2.imread(img_path)
                if image is None:
                    print(f"Warning: Unable to read image {filename}. Writing NaN...")
                    writer.writerow([filename, label, np.nan])
                    continue

                h, w = image.shape[:2]

                results = face_mesh.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

                if results.multi_face_landmarks:
                    landmarks = results.multi_face_landmarks[0].landmark

                    left_eye = [(int(landmarks[i].x * w), int(landmarks[i].y * h)) for i in LEFT_EYE_IDX]
                    right_eye = [(int(landmarks[i].x * w), int(landmarks[i].y * h)) for i in RIGHT_EYE_IDX]

                    left_ear = eye_aspect_ratio(left_eye)
                    right_ear = eye_aspect_ratio(right_eye)
                    avg_ear = (left_ear + right_ear) / 2.0

                    writer.writerow([filename, label, avg_ear])
                    print(f"{filename} | Label: {label} | Avg EAR: {avg_ear:.3f}")
                else:
                    # Instead of skipping, write NaN for missing EAR
                    writer.writerow([filename, label, np.nan])
                    print(f"{filename} | Label: {label} | EAR: NaN (no landmarks)")
